# Usage Stats – quick visuals

This notebook loads the CSV, cleans the header rows, converts numbers, and creates a few quick charts.

If your CSV file is in the same folder as this notebook, the code will work as-is.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

CSV_PATH = "Untitled spreadsheet - Usage Stats.csv"  # same folder as this .ipynb

raw = pd.read_csv(CSV_PATH, header=None)

# Row 0 is empty, row 1 contains the real headers, data starts at row 2
headers = raw.iloc[1].tolist()
df = raw.iloc[2:].copy()
df.columns = headers
df = df.reset_index(drop=True)

df.head()


In [ ]:
# Convert numeric columns (handles commas, blanks, etc.)
num_cols = ["No. pages", "% Pages", "No. of usage", "Average usage per page (Max for category)"]

def to_num(s):
    return pd.to_numeric(
        s.astype(str).str.replace(",", "", regex=False).str.strip(),
        errors="coerce"
    )

for c in num_cols:
    df[c] = to_num(df[c])

df.dtypes


In [ ]:
# Basic sanity checks
df.isna().sum()


In [ ]:
# Top 20 tags/widgets by total usage
top = df.sort_values("No. of usage", ascending=False).head(20)

plt.figure(figsize=(10, 6))
plt.barh(top["Tag / Widget"][::-1], top["No. of usage"][::-1])
plt.xlabel("No. of usage")
plt.ylabel("Tag / Widget")
plt.title("Top 20 by total usage")
plt.tight_layout()
plt.show()


In [ ]:
# Top 20 by average usage per page (skip very small samples if you want)
filtered = df[df["No. pages"] >= 50].sort_values("Average usage per page (Max for category)", ascending=False).head(20)

plt.figure(figsize=(10, 6))
plt.barh(filtered["Tag / Widget"][::-1], filtered["Average usage per page (Max for category)"][::-1])
plt.xlabel("Average usage per page")
plt.ylabel("Tag / Widget")
plt.title("Top 20 by average usage per page (No. pages ≥ 50)")
plt.tight_layout()
plt.show()


In [ ]:
# Relationship: pages vs avg usage per page
plt.figure(figsize=(8, 6))
plt.scatter(df["No. pages"], df["Average usage per page (Max for category)"])
plt.xlabel("No. pages")
plt.ylabel("Avg usage per page")
plt.title("Pages vs Avg usage per page")
plt.tight_layout()
plt.show()


In [ ]:
# Optional: save the cleaned data
df.to_csv("usage_stats_cleaned.csv", index=False)
print("Saved: usage_stats_cleaned.csv")
